In [ ]:
import pdfplumber
import pandas as pd
from pathlib import Path

In [ ]:
files = Path("EstadosDeCuenta")
msi = {}
spend = {}
output_folder = Path("debug_output")
    #Iterate every file in the Path
def is_msi_table(table):
    if not table or len(table) < 1:
        return False
    first_cell = str(table[0][0])
    return "COMPRAS Y CARGOS DIFERIDOS A MESES SIN INTERESES" in first_cell
def is_spend_table(table):
    if not table or len(table) < 1:
        return False
    first_cell = str(table[0][0])
    return "CARGOS,COMPRAS Y ABONOS REGULARES" in first_cell
for file in files.iterdir():
    if file.is_file():
        print(file)
    words = file.stem.split("_")
    month = words[-1]
    print(month)
    #Read the pdf in the file_path
    found_msi = False
    found_spent = False
    with pdfplumber.open(file) as pdf:
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()
            
            if not found_msi and "COMPRAS Y CARGOS DIFERIDOS A MESES SIN INTERESES" in text:
                print(f"msi found in page {page_num + 1}")
                setting = {
                    "vertical_strategy": "lines",
                    "horizontal_strategy": "lines",
                    "snap_tolerance": 5,    
                    "join_tolerance": 5,
                    "intersection_tolerance": 5,        
                }
                
                
                
                tables = page.extract_tables(table_settings=setting)
                if tables:
                    for table in tables:
                        if is_msi_table(table):    
                            #first dataframe with msi
                            df1 = pd.DataFrame(table[1:], columns=table[0])
                            df1 = df1.replace([r'\n', 'Tarjeta Digital',r'\*\*\*9552', 'Tarjeta titular: XXXXXXXXXXXX7445'], ' ', regex=True)
                            df1.columns.values[0] = month
                            msi[month] = df1
                            found_msi = True
                            break
                    if found_msi:
                        for table in tables:
                            if is_spend_table(table): 
                                print(f"spents found in page {page_num + 1}")
                                #Second dataframe with normal spends
                                df2 = pd.DataFrame(table[1:], columns=table[0])
                                df2 = df2.replace([r'\n', 'Tarjeta Digital', r'\*\*\*9552', 'Tarjeta titular: XXXXXXXXXXXX7445  '], ' ', regex=True)
                                df2.columns.values[0] = month
                                spend[month] = df2
                                found_spent = True
                                break    
                im = page.to_image(resolution=150)
                debug_image = im.debug_tablefinder(setting)
                save_path = output_folder / f"debug_{month}.png"
                debug_image.save(save_path)
            elif found_msi and not found_spent:
                if "CARGOS,COMPRAS Y ABONOS REGULARES(NO A MESES)" in text:
                    
                    setting = {
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                        "snap_tolerance": 5,    
                        "join_tolerance": 5,
                        "intersection_tolerance": 5,        
                    }
                    
                    tables = page.extract_tables(table_settings=setting)
                    
                    if tables:
                        for table in tables:
                            if is_spend_table(table):
                                print(f"spends found in page {page_num + 1}")
                                df2 = pd.DataFrame(table[1:], columns=table[0])
                                df2 = df2.replace([r'\n', 'Tarjeta Digital', r'\*\*\*9552', 'Tarjeta titular: XXXXXXXXXXXX7445  '], ' ', regex=True)
                                df2.columns.values[0] = month
                                spend[month] = df2
                                found_spent = True
                                break
                            
            if found_msi and found_spent:
                break

In [ ]:
spend["octubre"]
